Importing data

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

Importing CSVs

In [46]:
x_train=pd.read_csv(r'G:\Projects\Ground Water Level Estimator - Group Project\dataset\cleanedData\x_train.csv')
x_train.head()

,state_code,district_code,latitude,longitude,basin,sub_basin,level_diff,month,season_Monsoon,season_Post-Monsoon,season_Summer,season_Winter
0,6,68,29.97361,76.82778,Indus (Up To Border),Ghaghar And Others,3.10,3,0.0,0.0,1.0,0.0
1,21,355,20.32778,86.08583,Mahanadi,Mahanadi Lower,0.23,1,0.0,0.0,0.0,1.0
2,8,112,25.66667,74.01667,Ganga,Banas,0.80,11,0.0,1.0,0.0,0.0
3,29,524,16.53972,75.32972,Krishna,Krishna Upper,3.83,5,0.0,0.0,1.0,0.0
4,29,546,16.05000,77.21389,Krishna,Tungabhadra Lower,2.20,8,1.0,0.0,0.0,0.0


In [47]:
x_test=pd.read_csv(r'G:\Projects\Ground Water Level Estimator - Group Project\dataset\cleanedData\x_test.csv')
x_test.head()

,state_code,district_code,latitude,longitude,basin,sub_basin,level_diff,month,season_Monsoon,season_Post-Monsoon,season_Summer,season_Winter
0,36,507,17.38333,78.52083,Krishna,Krishna Lower,1.44,5,0.0,0.0,1.0,0.0
1,23,409,22.40750,77.84278,Narmada,Narmada Upper,0.52,1,0.0,0.0,0.0,1.0
2,23,416,24.14833,75.02917,Ganga,Chambal Upper,10.04,5,0.0,0.0,1.0,0.0
3,19,305,23.40000,86.97361,Ganga,Damodar,2.31,1,0.0,0.0,0.0,1.0
4,33,568,13.02500,80.25833,East Flowing Rivers Between Pennar And Kanyaku...,Palar And Other,-0.13,11,0.0,1.0,0.0,0.0


In [48]:
y_train=pd.read_csv(r'G:\Projects\Ground Water Level Estimator - Group Project\dataset\cleanedData\y_train.csv')
y_train.head()

,currentlevel
0,36.40
1,1.33
2,2.10
3,7.13
4,12.20


In [49]:
y_test=pd.read_csv(r'G:\Projects\Ground Water Level Estimator - Group Project\dataset\cleanedData\y_test.csv')
y_test.head()

,currentlevel
0,7.02
1,2.23
2,19.14
3,10.45
4,2.53


Shape Ideas

In [50]:
x_train.shape,x_test.shape

((440680, 12), (110170, 12))

In [51]:
y_train.shape,y_test.shape

((440680, 1), (110170, 1))

Describing

In [52]:
x_train.describe()

,state_code,district_code,latitude,longitude,level_diff,month,season_Monsoon,season_Post-Monsoon,season_Summer,season_Winter
count,440680.000000,440680.000000,440680.000000,440680.000000,440680.000000,440680.000000,440680.000000,440680.000000,440680.000000,440680.000000
mean,22.558004,397.033970,20.150001,79.306261,0.020212,6.149224,0.238284,0.250202,0.226697,0.284817
std,9.198099,169.356958,6.205204,4.764904,5.782382,3.804822,0.426034,0.433130,0.418696,0.451328
min,1.000000,1.000000,8.083330,68.675000,-276.580000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,312.000000,14.898957,75.995830,-1.300000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,23.000000,433.000000,21.345830,78.166670,0.300000,6.000000,0.000000,0.000000,0.000000,0.000000
75%,29.000000,535.000000,24.470560,82.230560,1.550000,10.000000,0.000000,1.000000,0.000000,1.000000
max,38.000000,704.000000,33.741110,96.133330,216.400000,12.000000,1.000000,1.000000,1.000000,1.000000


In [53]:
scaler=StandardScaler()

Preprocessing: Clean Up y_train and y_test Shape

In [54]:
y_train = y_train.squeeze()
y_test = y_test.squeeze()

Feature Scaling
- Use a ColumnTransformer to scale only numeric features,not one-hot encoded ones like season_Winter

In [55]:


numeric_features = ['state_code', 'district_code', 'latitude', 'longitude', 'level_diff', 'month']
categorical_features = ['basin', 'sub_basin']  # <-- These were causing issues

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
], remainder='passthrough')

x_train_processed = preprocessor.fit_transform(x_train)
x_test_processed = preprocessor.transform(x_test)


Train Multiple Regressors in a Loop

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

models = {
    "LinearRegression": LinearRegression(),
    "LassoCV": LassoCV(cv=5),
    "RidgeCV": RidgeCV(cv=5),
    "ElasticNetCV": ElasticNetCV(cv=5),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "AdaBoost": AdaBoostRegressor(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, random_state=42)
}

results = []

for name, model in models.items():
    model.fit(x_train_processed, y_train)
    y_pred = model.predict(x_test_processed)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    n, p = x_test.shape
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

    results.append({
        'Model': name,
        'MSE': mse,
        'MAE': mae,
        'R2': r2,
        'Adjusted R2': adj_r2
    })


Display Results

In [ ]:
results_df = pd.DataFrame(results).sort_values(by='R2', ascending=False)
results_df.style.background_gradient(cmap='YlGnBu')

Visual Comparison of Model Performance

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='R2', y='Model', data=results_df, palette='viridis')
plt.title('Model R² Comparison')
plt.xlabel('R² Score')
plt.ylabel('Model')
plt.grid(True)
plt.tight_layout()
plt.show()